### Problem 6-2 - Exploration work


Initial Problem, look at Lloyds Mirror effect in the vessel noise standard.
For a set of frequencies, look at variations in RL at the three hydrophone locations.

Vessel: h=2m, modeled as a point source
Hydrophones: h=27m, 58m, 100m

Plot a table of RL vs fc and h.

Model sound at d.  Just 2 distances involved in deep water.   Pressure release BC's mean reflection will be phase shifted by $\pi$.   

What is coefficient of model?    160 db re 1uPa^2 at 1m means that 20 log A/ref = 160 implies that A = Pref * 10^8 uPa RMS, so none RMS would be A/sqrt(2).  

This yields a value of A of sqrt(2)*100 Pa * m, or A = 144.4 Pa m

Now we can make a coherent sum of the 2 terms to get estimate for pressure at the different hydrophones / frequencies.





In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy import signal
import scipy.io as io
import os

In [22]:
# generate [n,fmin, fx, fmax] tuples, effectively in a 16 x 4 
# 1/3 octave bands, starting at 100 Hz
# each band increases by 2**1/3

fc = 100
f = [fc]
print("0, first 1/3 octave band: {:.3f}".format(fc))
for n in range(1,21):
    fc*=np.power(2,1/3)
    f.append(fc)
    print("{}, next 1/3 octave band: {:.3f}".format(n, fc))


0, first 1/3 octave band: 100.000
1, next 1/3 octave band: 125.992
2, next 1/3 octave band: 158.740
3, next 1/3 octave band: 200.000
4, next 1/3 octave band: 251.984
5, next 1/3 octave band: 317.480
6, next 1/3 octave band: 400.000
7, next 1/3 octave band: 503.968
8, next 1/3 octave band: 634.960
9, next 1/3 octave band: 800.000
10, next 1/3 octave band: 1007.937
11, next 1/3 octave band: 1269.921
12, next 1/3 octave band: 1600.000
13, next 1/3 octave band: 2015.874
14, next 1/3 octave band: 2539.842
15, next 1/3 octave band: 3200.000
16, next 1/3 octave band: 4031.747
17, next 1/3 octave band: 5079.683
18, next 1/3 octave band: 6400.000
19, next 1/3 octave band: 8063.495
20, next 1/3 octave band: 10159.367


In [14]:
# What is value of A in spherical point source model?
# 160 db = 20 log (Arms / Pref)
Arms = 1e-6 * 10**8
# So for model of real pressure, 
print("A = Arms * sqrt(2) = {:.3f} [Pa m]".format(Arms*np.sqrt(2)))
print("Arms = A/sqrt(2) = {:.1f} [Pa m]".format(Arms))

A = Arms * sqrt(2) = 141.421 [Pa m]
Arms = A/sqrt(2) = 100.0 [Pa m]


In [36]:
# Now do first calculation
# A    = 141.4 Pa m
# rho = 1025 kg/m^3
# c   = 1500 m/s
# k = 2 pi / lambda = 2 pi f / c
c = 1500
k = np.array(f)*2*np.pi/c 
A = 141.42 
d = 100           # distance (along x axis)
h = np.array([27,58,100])   # depth of hydrophone
z = 2             # depth of source

In [41]:
P = np.zeros((21,3), dtype=np.csingle)
for i in range(h.shape[0]):
    r1 = np.sqrt(d**2 + (z-h[i])**2)
    r2 = np.sqrt(d**2 + (-z-h[i])**2)
    P[:,i] = A*(np.exp(1j * k * r1)/r1 - np.exp(1j * k * r2)/r2 ) 

In [43]:
RL = 10 * np.log10(np.abs(0.5*P*np.conjugate(P))/1e-12)

In [64]:
# todo turn into print function
print("| Freq [Hz]\tk [1/m]\th1 = {} [m]\th2 = {} [m]\th3 = {} [m]\t\t|".format(h[0], h[1], h[2]))
for i in range(RL.shape[0]):
    if i <= 9:
        print("| {:.1f}\t\t{:.1f}\t{:.2f} [dB]\t{:.2f} [dB]\t{:.2f} [dB]\t|".format(
            f[i], k[i], RL[i,0], RL[i,1], RL[i,2]
        ))
    else:
        print("| {:.1f}\t{:.1f}\t{:.2f} [dB]\t{:.2f} [dB]\t{:.2f} [dB]\t|".format(
            f[i], k[i], RL[i,0], RL[i,1], RL[i,2]
        ))
    

| Freq [Hz]	k [1/m]	h1 = 27 [m]	h2 = 58 [m]	h3 = 100 [m]		|
| 100.0		0.4	112.43 [dB]	116.98 [dB]	117.95 [dB]	|
| 126.0		0.5	114.39 [dB]	118.83 [dB]	119.65 [dB]	|
| 158.7		0.7	116.34 [dB]	120.59 [dB]	121.16 [dB]	|
| 200.0		0.8	118.24 [dB]	122.20 [dB]	122.35 [dB]	|
| 252.0		1.1	120.08 [dB]	123.57 [dB]	122.98 [dB]	|
| 317.5		1.3	121.82 [dB]	124.52 [dB]	122.59 [dB]	|
| 400.0		1.7	123.40 [dB]	124.71 [dB]	119.88 [dB]	|
| 504.0		2.1	124.71 [dB]	123.39 [dB]	106.87 [dB]	|
| 635.0		2.7	125.57 [dB]	117.93 [dB]	118.29 [dB]	|
| 800.0		3.4	125.58 [dB]	111.57 [dB]	123.01 [dB]	|
| 1007.9	4.2	123.86 [dB]	123.74 [dB]	112.77 [dB]	|
| 1269.9	5.3	116.85 [dB]	122.94 [dB]	122.52 [dB]	|
| 1600.0	6.7	116.46 [dB]	117.37 [dB]	97.63 [dB]	|
| 2015.9	8.4	125.29 [dB]	122.98 [dB]	118.36 [dB]	|
| 2539.8	10.6	122.27 [dB]	124.30 [dB]	118.80 [dB]	|
| 3200.0	13.4	121.93 [dB]	122.52 [dB]	103.52 [dB]	|
| 4031.7	16.9	121.02 [dB]	124.27 [dB]	122.56 [dB]	|
| 5079.7	21.3	125.67 [dB]	120.32 [dB]	122.75 [dB]	|
| 6400.0	26.8	125.6